In [1]:
from markupsafe import escape
from selenium import webdriver
from thesentiment import sentiment
from collections import Counter
import re
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
from google_trans_new import google_translator
import numpy as np
import json

In [2]:
def get_tweet_data(card):
    """Extract data from tweet card"""
    username = card.find_element_by_xpath('.//span').text
    try:
        handle = card.find_element_by_xpath('.//span[contains(text(), "@")]').text
    except NoSuchElementException:
        return

    try:
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return

    comment = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    responding = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text

    text = comment + responding

    reply_cnt = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweet_cnt = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    like_cnt = card.find_element_by_xpath('.//div[@data-testid="like"]').text
    tweet = (username, handle, postdate, text, reply_cnt, retweet_cnt, like_cnt)
    return tweet


In [6]:
datanum=30
tweettt ='nike'
serv='normal'

#-------------------------------------

# connexion vers twitter
#----------------------------------------------------
path =r'D:\downloads\chromedriver_win324/chromedriver.exe'

driver = webdriver.Chrome(path)
driver.get("https://twitter.com/login")
sleep(4)
username = driver.find_element_by_xpath('//input[@name="username"]')
username.send_keys('afifzak79764501')
username.send_keys(Keys.RETURN)
sleep(4)
try:
    password = driver.find_element_by_xpath('//input[@name="password"]')
    password.send_keys('afifzakaria1997')
    password.send_keys(Keys.RETURN)
except:
    email = driver.find_element_by_xpath('//input[@name="text"]')
    email.send_keys('0629525883')
    email.send_keys(Keys.RETURN)
    sleep(4)
    password = driver.find_element_by_xpath('//input[@name="password"]')
    password.send_keys('afifzakaria1997')
    password.send_keys(Keys.RETURN)
sleep(8)


# ----------------------------------------------------

#query (entrer le nom du topic à rechercher dans l'input query)
# ----------------------------------------------------
search_input = driver.find_element_by_xpath('//input[@data-testid="SearchBox_Search_Input"]')
if serv=='normal':
    searchfiltre='min_faves:50  -filter:links filter:replies lang:en '
else:
    searchfiltre='min_faves:50  -filter:links filter:replies '
    valuemax=300
    valuemax1=datanum
    datanum=min(valuemax1,valuemax)
search_input.send_keys(searchfiltre+ tweettt)
search_input.send_keys(Keys.RETURN)
sleep(6)
try:
    driver.find_element_by_link_text('Latest').click()
except:
    sleep(3)
    driver.find_element_by_link_text('Récent').click()
# ----------------------------------------------------

#scraping and scrolling
# ----------------------------------------------------
data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True
while scrolling:
    page_cards = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
                if len(data) == datanum:
                    scrolling = False
                    break
    scroll_attempt = 0
    while True:
        # check scroll position
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(2)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            # end of scroll region
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(2)  # attempt another scroll
        else:
            last_position = curr_position
            break
    # close the web driver
    
driver.close()

# ----------------------------------------------------

#insertion des données dans un ensemble de données (dataset)
# ----------------------------------------------------
dr = pd.DataFrame(data, columns=['UserName', 'Handle', 'Timestamp', 'text', 'Cnts', 'Likes', 'Retweets'])

dic = dict()
for r in dr.keys():
    dic[r] = []
for r in dr.keys():
    for l in dr[r]:
        if l == '':
            dic[r].append(np.nan)
        elif l is None:
            dic[r].append(np.nan)
        else:
            dic[r].append(l)
df = pd.DataFrame()
for r in dr.keys():
    df[r] = dic[r]


# ----------------------------------------------------


#insertion des données avant prétraitement dans une collection mongodb

# ----------------------------------------------------


# ----------------------------------------------------

#préatraitement
# ----------------------------------------------------
lst = []
for l in df['text']:
    if ('Replying to' in l) and ('@' in l):
        d = l.split('@')[-1].lstrip().split(' ')
        s = ''
        for i in range(len(d)):
            if i >= 1:
                s = s + ' ' + d[i]
        lst.append(s)
    else:
        lst.append(l)

df['newtext'] = lst
df['newtext'] = df['newtext'].replace('\n', '').replace('@', '')



lst = []
for l in df['newtext']:
    d = ''.join([i for i in str(l) if (ord(i) >= 65 and ord(i) <= 90) or (
                ord(i) >= 97 and ord(i) <= 122) or i == ' ' or i.isdigit()])
    lst.append(d)
df['newtext2'] = lst
lst = []
for l in df['text']:
    if ('Replying to' in l) and ('@' in l):
        u = []
        d = l.replace('Replying to', '').strip().split('@')
        for i in range(len(d)):
            if i >= 1 and i < (len(d) - 1):
                u.append(d[i].replace('and', '').replace('\n', ''))
            else:
                if i == (len(d) - 1):
                    u.append(d[i].split(' ')[0].replace('and', '').replace('\n', ''))
        lst.append(u)
    else:
        lst.append(np.nan)

df['receiver'] = lst
df['Handle'] = df['Handle'].replace('@', '')



df.rename(columns={'UserName': 'username',
                   'Handle': 'nickname',
                   'newtext2': 'tweet', 'Timestamp': 'timestamp'},
          inplace=True)
df['timestamp'] = pd.to_datetime(df['timestamp']).dt.date
df = df[df['tweet'].notnull()]
df = df[df['tweet'] != '']
df = df.reset_index(drop=True)


#traduction si l'utilisateur a choisi le service "translated"
# ----------------------------------------------------

if serv=='translated':
    p = dict()
    p['translation'] = []
    p['culture'] = []

    for com in df['tweet']:

        translator = google_translator()
        try:
            p['translation'].append(translator.translate(com, lang_tgt='en'))
        except:
            p['translation'].append(com)

        try:
            if len(translator.detect(com)) > 1:
                p['culture'].append(translator.detect(com)[1])
            else:
                p['culture'].append('unknown')
        except:
            p['culture'].append('unknown')

    df['tweet_translated'] = p['translation']
    df['culture'] = p['culture']

# ----------------------------------------------------


#analyse des sentiments dans le cas du service " normal " et dans le cas du service " translated"
# ----------------------------------------------------

if serv=='normal':
    s = []
    for com in df['tweet']:
        s.append(sentiment(com.lower()))
else:
    s = []
    for com in df['tweet_translated']:
        s.append(sentiment(com.lower()))

dd = dict()
dd['class'] = []
dd['confidence'] = []
for elem in s:
    dd['class'].append(elem[0])
    dd['confidence'].append(elem[1])
df['class'] = dd['class']
df['confidence'] = dd['confidence']
df.drop(columns=['text', 'Cnts', 'Likes', 'Retweets', 'newtext'], inplace=True)
del(df['receiver'])
df

<ipython-input-6-d5bd9e4fec90>:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)
<ipython-input-6-d5bd9e4fec90>:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  username = driver.find_element_by_xpath('//input[@name="username"]')
<ipython-input-6-d5bd9e4fec90>:19: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  password = driver.find_element_by_xpath('//input[@name="password"]')
<ipython-input-6-d5bd9e4fec90>:23: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  email = driver.find_element_by_xpath('//input[@name="text"]')
<ipython-input-6-d5bd9e4fec90>:27: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  password = driver.find_element_by_xpath('//input[@name="password"]')
<ipython-input-6-d5bd9e4fec90>:37: De

,username,nickname,timestamp,tweet,class,confidence
0,kia haumaru to nono kiwis,@yoblackironman,2021-11-03,kia haumaru to nono kiwisyoblackironman5hEn rp...,pos,1.00
1,Samer Awada,@sneaker_plug_la,2021-11-03,Samer Awadasneakerplugla8hEn rponse SOLELINKS...,neg,1.00
2,Jay Baruchel,@BaruchelNDG,2021-11-03,Jay BaruchelBaruchelNDG8hI got a brand new pai...,neg,1.00
3,Tulumbacı,@ahmetcannguner,2021-11-03,Tulumbacahmetcannguner11hEn rponse alniopules...,neg,0.75
4,GDara / inactive like my bias,@badeth012,2021-11-03,GDara inactive like my biasbadeth01215hSandar...,pos,0.75
5,Tammy David,@tammydavid,2021-11-03,Tammy Davidtammydavid17hRooting for Uniqlo She...,pos,0.75
6,NotKatarina.eth,@not_katarina,2021-11-03,NotKatarinaethnotkatarina17h nike trademark to...,pos,0.75
7,Metahero Club,@metaheroclub,2021-11-03,Metahero Clubmetaheroclub18hEn rponse Metaher...,pos,0.75
8,Metahero.io,@Metahero_io,2021-11-03,MetaheroioMetaheroio18hEn rponse RobGryn Nike...,pos,1.00
9,Blockworks,@Blockworks_,2021-11-02,BlockworksBlockworks2 nov2 REPORTED IN CRYPTO ...,neg,0.75
